In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def angular_resolution(wavelength, diameter):
    """
    Return the expected Angular Resolution
    
    Units are in radians
    """
    angular_resolution = 1.22 * (wavelength / diameter)
    return angular_resolution

def sample_taken(frequency, move_speed, angular_size):
    """
    frequency (sampling value): sample collection rate. (hertz)
    move_speed: angle movement in the azimuth direction. (deg/azimuth)
    angular_size: size of the source in the sky.
    
    1 degree in the sky refers to 3600 arcseconds
    """
    spacing = (move_speed / frequency) # (degree / azimuth)
    arc_seconds = 3600 * spacing # converting out spacing into arcseconds
    num_samples = angular_size / arc_seconds
    return num_samples

In [4]:
DIAMETERS = np.array([1.6])
WAVELENGTH_mc = [175, 250, 350] # microns
WAVELENGTH = np.array([wv * (1e-6) for wv in WAVELENGTH_mc]) # convert metres


angular_resolutions = {}
degrees = {}
for dia in DIAMETERS:
    for wvl in WAVELENGTH:
        ang_res = angular_resolution(wavelength = wvl, diameter = dia)
        angular_resolutions[str(round(dia, 7)) + " " + str(round(wvl, 7))] = round(angular_resolution(wvl, dia), 8)
        degrees[str(round(dia, 7)) + " " + str(round(wvl, 7))] = round(angular_resolution(wvl, dia) * (180/np.pi), 8)
        
print("Angular Resolutions in Radians")        
print(angular_resolutions)

print("\nAngular Resolutions in Degrees")
print(degrees)

Angular Resolutions in Radians
{'1.6 0.000175': 0.00013344, '1.6 0.00025': 0.00019062, '1.6 0.00035': 0.00026687}

Angular Resolutions in Degrees
{'1.6 0.000175': 0.00764541, '1.6 0.00025': 0.01092201, '1.6 0.00035': 0.01529081}


In [5]:
# If we assume a movement speed of 0.1 arc degree / second, then what is the scanning speed?

arc_seconds = []

print("Converting our Angular Resolutions in Degrees to Arcseconds...")
for id in degrees:
    print("for", degrees[id], "degrees it takes:", round(3600*degrees[id], 7), "arcseconds")
    arc_seconds.append(3600*degrees[id])

Converting our Angular Resolutions in Degrees to Arcseconds...
for 0.00764541 degrees it takes: 27.523476 arcseconds
for 0.01092201 degrees it takes: 39.319236 arcseconds
for 0.01529081 degrees it takes: 55.046916 arcseconds


In [6]:
# We know the speed of our scan to be 0.1 deg/sec -> 360 arcsecond/second
# if the angular resolution is 35 arc seconds, then the time it takes to cross the FWHM of the beam once is
# 35.642556 / 360 = 0.099 seconds
# If we want to take 4 samples during the time it takes to cross the beam once, then 0.099 / 4 = 0.00275 seconds
# which is how fast we would need to sample if we want to TAKE 4 samples per beam crossing.
# so....

beam_cross_time = []

for arcsec in arc_seconds:
    # we divide by 4 because we want at least 4 samples per beam crossing.
    beam_cross_time.append((arcsec / 360) / 3)

minimal_sampling_rate = {}

index = 0
for id in degrees:
    minimal_sampling_rate[id] = (1/beam_cross_time[index])
    index += 1
    

In [7]:
# Diplay our Angular Resolutions Clearly

print("\nAngular Resolution (Degree)")
for id in degrees:
    print("for DIAMETER and WAVELENGTH(metres) of:", id, "our angular resolution in degrees is", str(degrees[id]), "degrees")
    
    
print("\nAngular Resolution (Arc Seconds)")
for id in arc_seconds:
    print("for DIAMETER and WAVELENGTH(metres) of:", round(id, 7), "our angular resolution in arc seconds is", str(round(id, 7)), "degrees")
        
# Display our Minimal sampling rate Clearly


print("\nSampling Rate")
for id in minimal_sampling_rate:
    print("for DIAMETER and WAVELENGTH of:", id, "our sampling rate is", str(minimal_sampling_rate[id]), "hz")


Angular Resolution (Degree)
for DIAMETER and WAVELENGTH(metres) of: 1.6 0.000175 our angular resolution in degrees is 0.00764541 degrees
for DIAMETER and WAVELENGTH(metres) of: 1.6 0.00025 our angular resolution in degrees is 0.01092201 degrees
for DIAMETER and WAVELENGTH(metres) of: 1.6 0.00035 our angular resolution in degrees is 0.01529081 degrees

Angular Resolution (Arc Seconds)
for DIAMETER and WAVELENGTH(metres) of: 27.523476 our angular resolution in arc seconds is 27.523476 degrees
for DIAMETER and WAVELENGTH(metres) of: 39.319236 our angular resolution in arc seconds is 39.319236 degrees
for DIAMETER and WAVELENGTH(metres) of: 55.046916 our angular resolution in arc seconds is 55.046916 degrees

Sampling Rate
for DIAMETER and WAVELENGTH of: 1.6 0.000175 our sampling rate is 39.23922981239724 hz
for DIAMETER and WAVELENGTH of: 1.6 0.00025 our sampling rate is 27.467471646702396 hz
for DIAMETER and WAVELENGTH of: 1.6 0.00035 our sampling rate is 19.61962773718331 hz


1600 - 175 microns
850 - 250 microns
490 - 350 microns

Using this information, calculate the minimal sampling rate for each frequency band (sample rate will depend on the beam size). 
Then calculate, 

WE ASSUME THAT OUR NORMAL FREQUENCY RATE IS 100 HZ.

Need to know the beam sizes.
    - 27 arcseconds (175 microns)
    - 39 arcseconds (250 microns)
    - 54 arcseconds (350 microns)


e.g. 250 micron is 80hz (assume factor of 5 compression from rice algorithm), calculate from a single detector, 
what the data rate will be.

Knowing this, we can sum the data rate for all detectors for each bandwidth.
"""
print()

In [17]:
def minimal_bandwidth(num_detectors, frequency, comp_factor=4.69):
    """
    What is the minimal bandwidth?
    """
    return num_detectors * frequency * 32 / comp_factor

In [18]:
min_frequencies = [39.239 * 2, 27.467 * 2, 19.619 * 2]  #175, 250, 350 microns respectively
num_detectors = [1600, 850 , 490] # assuming we switch the number of detectors around...

total_downlink = 0

for i in range(3):
    print((minimal_bandwidth(num_detectors[i], min_frequencies[i])))
    total_downlink += (minimal_bandwidth(num_detectors[i], min_frequencies[i]))
    
print("Our minimal downlink speed is therefore: " + str(total_downlink / 1e6), "mbits / second")

856732.1108742002
318593.7739872068
131183.76119402982
Our minimal downlink speed is therefore: 1.3065096460554368 mbits / second


In [13]:
# Nyquist sampling 
print(min_frequencies)

[78.478, 54.934, 39.238]
